In [ ]:
import numpy as np
import torch

In [ ]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device=",device)

device= cuda


### 1. model, tokenizer

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 72.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 91.8 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, BertForSequenceClassification

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("textattack/bert-base-uncased-yelp-polarity")
model = BertForSequenceClassification.from_pretrained("textattack/bert-base-uncased-yelp-polarity")

In [ ]:
from transformers import BertTokenizer

In [ ]:
inputs1 = tokenizer("I like it",return_tensors="pt")

with torch.no_grad():
    logits1 = model(**inputs1).logits

predicted_class_id1 = logits1.argmax().item()

model.config.id2label[predicted_class_id1]

'LABEL_1'

In [ ]:
predicted_class_id1

1

In [ ]:
logits1

tensor([[-3.2114,  3.6183]])

### 2. the dataset

In [ ]:
!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d team-ai/spam-text-message-classification

100% 208k/208k [00:00<00:00, 733kB/s]
100% 208k/208k [00:00<00:00, 732kB/s]


In [ ]:
!unzip /content/spam-text-message-classification.zip

Archive:  /content/spam-text-message-classification.zip
  inflating: SPAM text message 20170820 - Data.csv  


In [ ]:
import pandas as pd

data = pd.read_csv('SPAM text message 20170820 - Data.csv')
data.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
label_mapping = {'ham': 1, 'spam': 0}
data['Label'] = data['Category'].map(label_mapping)

data.head()

,Category,Message,Label
0,ham,"Go until jurong point, crazy.. Available only ...",1
1,ham,Ok lar... Joking wif u oni...,1
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,0
3,ham,U dun say so early hor... U c already then say...,1
4,ham,"Nah I don't think he goes to usf, he lives aro...",1


In [ ]:
data.isnull().values.any()

False

In [ ]:
Messages = data.Message.values
Messages

array(['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
       'Ok lar... Joking wif u oni...',
       "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
       ..., 'Pity, * was in mood for that. So...any other suggestions?',
       "The guy did some bitching but I acted like i'd be interested in buying something else next week and he gave it to us for free",
       'Rofl. Its true to its name'], dtype=object)

In [ ]:
Labels = data.Label.values
Labels


array([1, 1, 0, ..., 1, 1, 1])

### 3. Text Tokenization

In [ ]:
Messages[1]

'Ok lar... Joking wif u oni...'

In [ ]:
tokenizer(Messages[1])

{'input_ids': [101, 7929, 2474, 2099, 1012, 1012, 1012, 16644, 15536, 2546, 1057, 2006, 2072, 1012, 1012, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
print(tokenizer.tokenize(Messages[1]))
print(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(Messages[1])))

['ok', 'la', '##r', '.', '.', '.', 'joking', 'wi', '##f', 'u', 'on', '##i', '.', '.', '.']
[7929, 2474, 2099, 1012, 1012, 1012, 16644, 15536, 2546, 1057, 2006, 2072, 1012, 1012, 1012]


In [ ]:
max_len=0                #to make sure max_len will be updated every time
for Mess in Messages:
    input_ids = tokenizer.encode(Mess, add_special_tokens=True)       #this step is to roughly calcuate input_id first: encode and add CLS and SEP
    max_len = max(max_len, len(input_ids))                                #then find out the max length of them for later use      
print(max_len)             #the output shows that the max_len here is 238--it is okay since the maximum length in BERT is 512

238


In [ ]:
input_ids = []
attention_masks = []

for Mess in Messages:
    encoded = tokenizer.encode_plus(Mess,add_special_tokens = True,padding = 'max_length',max_length=max_len,return_attention_mask = True,return_tensors = 'pt')
                                           #saw above                     #fill to max length calcuated above                               #return tensors as pytorch  
    
    input_ids.append(encoded['input_ids'])  #put every input_id output in the empty list 
    attention_masks.append(encoded['attention_mask'])  #similarily   

In [ ]:
#check the list--input_ids
print(len(input_ids))

print(input_ids[0].shape, input_ids[2].shape)

5572
torch.Size([1, 238]) torch.Size([1, 238])


In [ ]:
#take a closer look
print(input_ids[1])    #compare with the tokenizer(Messages[1]), common-->non-letters are filled; difference-->pad to max length in the dataset

tensor([[  101,  7929,  2474,  2099,  1012,  1012,  1012, 16644, 15536,  2546,
          1057,  2006,  2072,  1012,  1012,  1012,   102,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,  

In [ ]:
#now for this list, concatenate all tensors together 
input_ids = torch.cat(input_ids, dim=0)
input_ids.shape

torch.Size([5572, 238])

In [ ]:
#do the same thing for "Labels" list
Labels = torch.tensor(Labels)
Labels.shape

torch.Size([5572])

In [ ]:
#do the same thing for masks list, for later use
attention_masks = torch.cat(attention_masks, dim=0)
attention_masks.shape

torch.Size([5572, 238])

### 4. TensorDataset, DataLoader 

In [ ]:
from torch.utils.data import TensorDataset, random_split
from sklearn.model_selection import train_test_split

In [ ]:
dataset = TensorDataset(input_ids, attention_masks, Labels)

In [ ]:
train, test = train_test_split(dataset, test_size=0.2, random_state=42)

In [ ]:
abs(len(train) - 0.8*len(dataset)) <= 1    #verify the train and test size

True

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

train_dataloader = DataLoader(train,sampler = RandomSampler(train), batch_size = 32)   
                                    #rather than being processed in order, the data each time are sampled randomly without replacement  

test_dataloader = DataLoader(test,sampler = SequentialSampler(test),batch_size = 32)
                                    #same method as above

In [ ]:
import torch.nn as nn

# in the beginning I load the model already
# model = BertForSequenceClassification.from_pretrained("textattack/bert-base-uncased-yelp-polarity")
model.config.num_Labels = 2  
model.classifier = nn.Linear(model.config.hidden_size, model.config.num_Labels)

In [ ]:
model = model.to(device)

In [ ]:
import numpy as np

In [ ]:
CE=nn.CrossEntropyLoss()

import torch.optim as optim
optimizer=optim.Adam(model.parameters(),lr=2e-5)


from sklearn.metrics import accuracy_score
def flat_accuracy(preds,labels):
    pred_flat=preds.detach().cpu().numpy().argmax(preds,axis=1).flatten()
    labels_flat=labels.detach().cpu().numpy().flatten()
    return accuracy_score(labels_flat,pred_flat)

### 5. Feed 

In [ ]:
for epoch in range(10):   
    train_loss = 0.0
    train_acc=0.0
    correct=0
    total=0
    for i, data in enumerate(train_dataloader):
        input_ids = data[0].to(device)  # Accessing the 'input_ids' tensor from the batch_data dictionary
        attention_masks = data[1].to(device)  # Accessing the 'attention_mask' tensor from the batch_data dictionary
        labels = data[2].to(device) 
        
        optimizer.zero_grad()
        
        outputs=model(input_ids,attention_masks).logits
        
        loss=CE(outputs, labels)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
   
    accuracy = correct / total
    print(f"{epoch+1}/10: Loss is {train_loss/(i+1)}; Accuracy is {accuracy}")

1/10: Loss is 0.2282802727139954; Accuracy is 0.9257347991922819
2/10: Loss is 0.039498357295191716; Accuracy is 0.9896791563832175
3/10: Loss is 0.00996504940287975; Accuracy is 0.9975319721785955
4/10: Loss is 0.0043680495888111185; Accuracy is 0.9988781691720888
5/10: Loss is 0.0021242115836815046; Accuracy is 0.9997756338344178
6/10: Loss is 0.0018934681497090164; Accuracy is 0.9997756338344178
7/10: Loss is 0.0011553353878428294; Accuracy is 0.9997756338344178
8/10: Loss is 0.0006246296831575039; Accuracy is 0.9997756338344178
9/10: Loss is 0.0005321831649780506; Accuracy is 0.9997756338344178
10/10: Loss is 0.00046256964994557037; Accuracy is 0.9997756338344178
